In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from data_frame import DataFrame
from tensorflow.keras.layers import Embedding, Dense, LSTM, Conv1D, Activation, Add, Multiply, Lambda, Input, Concatenate
from tensorflow.keras.models import Model
from utils import lstm_layer, time_distributed_dense_layer, dense_layer, wavenet
import tabnet_model

In [50]:
data_dir = '../data/interim'
data_cols = [
            'user_id',
            'product_id',
            'aisle_id',
            'department_id',
            # 'is_ordered_history',
            # 'index_in_order_history',
            # 'order_dow_history',
            # 'order_hour_history',
            # 'days_since_prior_order_history',
            # 'order_size_history',
            # 'reorder_size_history',
            # 'order_number_history',
            # 'history_length',
            # 'product_name',
            # 'product_name_length',
            # 'eval_set',
            'label'
        ]

data = { i : np.load(os.path.join(data_dir, '{}.npy'.format(i)), mmap_mode='r') for i in data_cols }
# test_df = DataFrame(columns=data_cols, data=data)
                        
# print(test_df.shapes())
# print('loaded data')

# train_df, val_df = test_df.train_test_split(train_size=0.8)

# print('train size', len(train_df))
# print('val size', len(val_df))
# print('test size', len(test_df))

In [52]:
for k in data:
    df = pd.DataFrame(data[k])
    print()
    print(k)
    print("Shape: ", k)
    print("Dtype: ", df.dtypes)
    print("Row: ", df.head(1))


user_id
Shape:  user_id
Dtype:  0    int32
dtype: object
Row:     0
0  1

product_id
Shape:  product_id
Dtype:  0    int32
dtype: object
Row:         0
0  17122

aisle_id
Shape:  aisle_id
Dtype:  0    int16
dtype: object
Row:      0
0  24

department_id
Shape:  department_id
Dtype:  0    int8
dtype: object
Row:     0
0  4

label
Shape:  label
Dtype:  0    int8
dtype: object
Row:     0
0  0


In [ ]:
data_cols.remove('label')

In [57]:
# Input layers
user_id = Input(shape=(), dtype=tf.int32, name='user_id')
product_id = Input(shape=(), dtype=tf.int32, name='product_id')
aisle_id = Input(shape=(), dtype=tf.int32, name='aisle_id')
department_id = Input(shape=(), dtype=tf.int32, name='department_id')

In [61]:
inputs = [user_id, product_id, aisle_id, department_id]
x = tabnet_model.TabNetEncoder(**tabnet_params)(inputs)
output = Dense(1)(x)

model = Model(inputs, output)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'TensorShape'

In [59]:
tabnet_params = {
        "decision_dim": 16, 
        "attention_dim": 16, 
        "n_steps": 5, 
        "n_shared_glus": 2, 
        "n_dependent_glus": 2, 
        "relaxation_factor": 1.5, 
        "epsilon": 1e-15, 
        "virtual_batch_size": None, 
        "momentum": 0.98, 
        "mask_type": "entmax", 
        "lambda_sparse": 1e-4, 
}

In [ ]:
# Keras model using Functional API
inputs = create_keras_input_layer(data_cols)
x = encode_features(inputs, feature_names, 
                    cat_str_feature_names, cat_int_feature_names, cat_embed_dims, 
                    train_ds)
x = tf.keras.layers.Concatenate()(x)
x = tabnet_model.TabNetEncoder(**tabnet_params)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs, output)